In [4]:
from pathlib import Path
from PIL import Image, ImageFilter
import torch
import torchvision.transforms as transforms
import numpy as np
import random
import os

# 设置文件夹路径
noise_dirs = [
    './split_data/test/Agriculture', './split_data/test/Airport',
    './split_data/test/Beach', './split_data/test/City', './split_data/test/Desert',
    './split_data/train/Agriculture', './split_data/train/Airport',
    './split_data/train/Beach', './split_data/train/City', './split_data/train/Desert',
    './split_data/val/Agriculture', './split_data/val/Airport',
    './split_data/val/Beach', './split_data/val/City', './split_data/val/Desert'
]

occlude_dirs = [
    './split_data/test/Forest', './split_data/test/Grassland',
    './split_data/test/Highway', './split_data/test/Lake', './split_data/test/Mountain',
    './split_data/train/Forest', './split_data/train/Grassland',
    './split_data/train/Highway', './split_data/train/Lake', './split_data/train/Mountain',
    './split_data/val/Forest', './split_data/val/Grassland',
    './split_data/val/Highway', './split_data/val/Lake', './split_data/val/Mountain'
]

blur_dirs = [
    './split_data/test/Parking', './split_data/test/Port',
    './split_data/test/Railway', './split_data/test/Residential', './split_data/test/River',
    './split_data/train/Parking', './split_data/train/Port',
    './split_data/train/Railway', './split_data/train/Residential', './split_data/train/River',
    './split_data/val/Parking', './split_data/val/Port',
    './split_data/val/Railway', './split_data/val/Residential', './split_data/val/River'
]

output_base = './split_data_Perturbed'

# 图像处理函数
def add_noise(image_tensor):
    return torch.clamp(image_tensor + torch.randn_like(image_tensor) * 0.1, 0, 1)



def occlude_image(image_tensor):
    """
    随机遮挡图像中心附近的区域，遮挡面积为图像的1/4。
    假设图像尺寸为256x256。
    """
    c, h, w = image_tensor.shape
    occ_h, occ_w = h // 2, w // 2  # 遮挡区域为图像宽高的一半

    # 随机选择遮挡中心点位置，确保遮挡区域不越界
    center_y = random.randint(occ_h // 2, h - occ_h // 2)
    center_x = random.randint(occ_w // 2, w - occ_w // 2)

    # 计算遮挡边界
    start_y = center_y - occ_h // 2
    end_y = start_y + occ_h
    start_x = center_x - occ_w // 2
    end_x = start_x + occ_w

    # 创建遮挡图像
    occluded = image_tensor.clone()
    occluded[:, start_y:end_y, start_x:end_x] = 0  # 设置遮挡区域为黑色
    return occluded


def blur_image(image_pil):
    return image_pil.filter(ImageFilter.GaussianBlur(radius=2))

# 图像转换
to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()

# 处理函数
def process_images(dirs, process_fn, use_tensor=True):
    for dir_path in dirs:
        input_dir = Path(dir_path)
        # 提取相对路径的两级目录名，如 train/Agriculture
        parts = input_dir.parts[-2:]  # e.g., ['train', 'Agriculture']
        output_dir = Path(output_base) / parts[0] / parts[1]
        output_dir.mkdir(parents=True, exist_ok=True)

        for img_path in input_dir.glob('*.jpg'):
            image = Image.open(img_path).convert('RGB')
            if use_tensor:
                image_tensor = to_tensor(image)
                processed_tensor = process_fn(image_tensor)
                processed_image = to_pil(processed_tensor)
            else:
                processed_image = process_fn(image)

            output_path = output_dir / img_path.name
            processed_image.save(output_path)


# 开始处理三类图像
process_images(noise_dirs, add_noise, use_tensor=True)
process_images(occlude_dirs, occlude_image, use_tensor=True)
process_images(blur_dirs, blur_image, use_tensor=False)

print("图像处理完成！")


图像处理完成！
